```
         Copyright Rein Halbersma 2018-2021.
Distributed under the Boost Software License, Version 1.0.
   (See accompanying file LICENSE_1_0.txt or copy at
         http://www.boost.org/LICENSE_1_0.txt)
```

# View Top Row, Open Flag Winning Game files at Gravon Stratego Archive site

##  1. An adjusted version of Dobby125's blog post

On April 10th, 2017, [stratego.com](http://www.stratego.com/play/) user [Dobby125](http://forum.stratego.com/user/873-dobby125/) wrote an insightful [blog post](http://stratego-tips.blogspot.com/2017/04/top-row-open-flag-winning-game-files-at.html) (announced in this [forum post](http://forum.stratego.com/topic/357378-strategy-question-findingavoiding-bombs-at-the-end-of-games/?p=431925)) showing tables with winning Stratego games with an open flag on the front row. We reproduce these tables below, with the following adjustments:

1. We not only include the 85K setups in .xml format, but also the 17K setups in .gsn format. 
2. We not only provide the number of games and wins, but also the draw and loss counts. 
3. We summarize the average percentage score (win = 1.0, draw = 0.5 and loss = 0.0 points).
4. We not only tabulate by player (red or blue), but also by flag side (left or right).
5. We sort the flag positions in order of score percentage.
6. We sort the filenames of games with a winning front row flag on their dates.
7. We archive the various games in .zip files viewable with the [Gravon game player](http://www.gravon.de/gravon/stratego/strados2.jsp).

Note that the table cells in this notebook are not correctly aligned when rendered on GitHub.

In [1]:
from IPython.display import display
import pandas as pd

import gravon.package as pkg
import gravon.archive as archive
import gravon.pattern as pattern
from gravon.stats import league_table

db_setups = pkg.load_dataset('db_setups')

In [2]:
front_row = {
    'Outside Wings': (
        """
        F.........
        ..........
        ..........
        ..........
        """
    ),
    'Inside Wings': (
        """
        .F........
        ..........
        ..........
        ..........
        """
    ),
    'Center Squares': (
        """
        ....F.....
        ..........
        ..........
        ..........
        """
    )
}

flag_fmt = {
    'flag columns': pd.CategoricalDtype(categories=front_row.keys())
}

score_fmt = {
    'Score': '{:.1%}'
}

In [3]:
classic = db_setups.query('type == "classic"')

df = (pd
    .concat([
        pattern.match(classic, flag_pattern, mirrored=True).assign(**{'flag columns': flag_columns})
        for flag_columns, flag_pattern in front_row.items()
    ])
    .rename(columns={'match_type': 'flag side'})
    .replace({'flag side': { 
        'identity': 'left', 
        'mirrored': 'right'
    }})
    .sort_values('gid')
    .astype(dtype=flag_fmt)
)

In [4]:
display(df
    .groupby(['flag columns', 'player'])
    .agg(**league_table)
    .style.format(score_fmt)
)
display(df
    .groupby(['flag columns', 'flag side'])
    .agg(**league_table)
    .style.format(score_fmt)
)

In [5]:
for flag_columns in front_row.keys():
    files = (df
        .query('result == "win" & `flag columns` == @flag_columns')
        .filename
        .to_list()
    )
    print(f'The {len(files):,} files below were the winning games with flags on the top row in the {flag_columns.lower()}:')
    print('\n'.join(files), '\n')
    archive.make_zip(files, 'winning_front_row_flag_in_' + flag_columns.lower().replace(' ', '_'))

The 125 files below were the winning games with flags on the top row in the outside wings:
classic.2003.05.15.107.gsn
classic.2003.05.15.191.gsn
classic.2003.05.19.69.gsn
classic.2003.05.24.244.gsn
classic.2003.06.12.560.gsn
classic.2003.09.09.2357.gsn
classic.2003.09.24.5009.gsn
classic.2003.09.28.4108.gsn
classic.2003.10.12.4939.gsn
classic.2003.10.19.4992.gsn
classic.2003.10.28.5396.gsn
classic-2005.9-433.xml
classic-2005.9-1694.xml
classic-2005.9-2492.xml
classic-2005.9-2596.xml
classic-2005.9-2886.xml
classic-2005.9-4194.xml
classic-2005.9-6431.xml
classic-2005.9-6933.xml
classic-2005.10-622.xml
classic-2005.10-832.xml
classic-2005.10-1263.xml
classic-2005.10-1955.xml
classic-2005.10-2816.xml
classic-2005.10-3510.xml
classic-2005.10-4479.xml
classic-2005.10-4521.xml
classic-2005.10-5217.xml
classic-2005.10-5659.xml
classic-2005.10-5716.xml
classic-2005.10-5895.xml
classic-2005.10-6141.xml
classic-2005.10-6530.xml
classic-2005.10-6854.xml
classic-2005.10-6972.xml
classic-2005.10-77

## 2. The original version of Dobby125's blog post

Below we reproduce the [original tables](http://stratego-tips.blogspot.com/2017/04/top-row-open-flag-winning-game-files-at.html) as faithfully as possible. Note that we find slightly more winning games for all three flag columns.

In [6]:
keys = [ 'Center Squares', 'Outside Wings', 'Inside Wings' ]
front_row = { k: front_row[k] for k in keys }
flag_fmt = {
    'flag columns': pd.CategoricalDtype(categories=front_row.keys())
}
table_fmt = {
    'Total Wins'    : ('win', 'sum'  ),
    'Total Games'   : ('win', 'count'),
    'Win Percentage': ('win', 'mean' )
}
win_fmt = {
    'Win Percentage': '{:.2%}'
}

In [7]:
classic_xml = db_setups.query('type == "classic" & ext == ".xml"')

df = (pd
    .concat([
        pattern.match(classic_xml, flag_pattern, mirrored=True).assign(**{'flag columns': flag_columns})
        for flag_columns, flag_pattern in front_row.items()
    ])
    .astype(dtype=flag_fmt)
)

In [8]:
display(df
    .groupby(['flag columns', 'player'])
    .agg(**table_fmt)
    .style.format(win_fmt)
)

In [9]:
for flag_columns in front_row.keys():
    files = (df
        .query('result == "win" & `flag columns` == @flag_columns')
        .filename
        .sort_values()
        .to_list()
    )
    print(f'The files below were the winning games with flags on the top row in the {flag_columns.lower()}:')
    print('\n'.join(files), '\n')

The files below were the winning games with flags on the top row in the center squares:
classic-2005.10-3351.xml
classic-2005.11-7392.xml
classic-2006.0-313.xml
classic-2006.2-4060.xml
classic-2006.2-7249.xml
classic-2006.4-1615.xml
classic-2006.4-5912.xml
classic-2014.10-2664.xml
classic-2014.5-138.xml 

The files below were the winning games with flags on the top row in the outside wings:
classic-2005.10-1263.xml
classic-2005.10-1955.xml
classic-2005.10-2816.xml
classic-2005.10-3510.xml
classic-2005.10-4479.xml
classic-2005.10-4521.xml
classic-2005.10-5217.xml
classic-2005.10-5659.xml
classic-2005.10-5716.xml
classic-2005.10-5895.xml
classic-2005.10-6141.xml
classic-2005.10-622.xml
classic-2005.10-6530.xml
classic-2005.10-6854.xml
classic-2005.10-6972.xml
classic-2005.10-7710.xml
classic-2005.10-7795.xml
classic-2005.10-8249.xml
classic-2005.10-832.xml
classic-2005.11-1299.xml
classic-2005.11-1787.xml
classic-2005.11-2067.xml
classic-2005.11-2573.xml
classic-2005.11-3241.xml
classic-

## 3. Earlier forum posts

### a) Center squares

On April 9th, 2017, [stratego.com](http://www.stratego.com/play/) user [Dobby125](http://forum.stratego.com/user/873-dobby125/) wrote a [forum post](http://forum.stratego.com/topic/357378-strategy-question-findingavoiding-bombs-at-the-end-of-games/?p=431876) listing the number of winning front row flags in total, for the center squares and by player (red or blue). The .xml filenames for the winning front row flags on the center squares were also listed.

In [10]:
display(df
    .groupby(lambda _: '')
    .agg(**table_fmt)
    .style.format(win_fmt)
)
display(df
    .query('`flag columns` == "Center Squares"')
    .groupby('flag columns', observed=True)
    .agg(**table_fmt)
    .style.format(win_fmt)
)
display(df
    .groupby('player')
    .agg(**table_fmt)
    .style.format(win_fmt)
)

,Total Wins,Total Games,Win Percentage
,136,446,30.49%


,Total Wins,Total Games,Win Percentage
flag columns,,,
Center Squares,9,69,13.04%


,Total Wins,Total Games,Win Percentage
player,,,
red,70,238,29.41%
blue,66,208,31.73%


In [11]:
files = (df
    .query('result == "win" & `flag columns` == "Center Squares"')
    .filename
    .sort_values()
    .to_list()
)
quoted_files = [
    f'\'{file}\''
    for file in files
]
print(f'Winning games center flag: {", ".join(quoted_files)}')

Winning games center flag: 'classic-2005.10-3351.xml', 'classic-2005.11-7392.xml', 'classic-2006.0-313.xml', 'classic-2006.2-4060.xml', 'classic-2006.2-7249.xml', 'classic-2006.4-1615.xml', 'classic-2006.4-5912.xml', 'classic-2014.10-2664.xml', 'classic-2014.5-138.xml'


### b) Fun and interesting games

The last game on the above list, **classic-2014.5-138.xml**, was pointed out as a fun game to watch.

On April 11th, 2017, [stratego.com](http://www.stratego.com/play/) user [Dobby125](http://forum.stratego.com/user/873-dobby125/) wrote a [forum post](http://forum.stratego.com/topic/357378-strategy-question-findingavoiding-bombs-at-the-end-of-games/?p=431957) pointing out the game **2005.11-6645.xml** (which features a winning front row flag on the outside wings) as an interesting game to watch.